### Adding a interactive browser control.

In [1]:
import io
import json
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

tree_name = pd.read_excel('Data_Sources/Puu_nimetused_EE_ENG.xlsx')
relative_heights = pd.read_excel('Data_Sources/Suhtelised_tugikõrgused.xlsx')
log_volume_distribution = pd.read_excel('Data_Sources/Mahutabel.xlsx')


# Abifunktsioon väljade loomiseks
def bf(desc, value=0.0, minv=0.0, width='340px'):
    return widgets.BoundedFloatText(
        value=value, min=minv, description=desc,
        style={'description_width':'initial'},
        layout=widgets.Layout(width=width)
    )

# Vidinad
show_details     = widgets.Checkbox(value=False, description="Näita detailseid tabeleid (hinnakiri, sortimendid)", indent=False)
komplekt_widget  = bf('Kompleksteenus(€/tm):', 15)
transport_widget = bf('Transport (€/tm):', 6)
alghind_widget   = bf('Alghinna(%):', 10)

# Puiduhindade väljad (genereeritakse nimekirjast)
price_names = [
    "Ma palk","Ku palk","Ks palk/pakk","Lv palk",
    "Ma peenpalk","Ku peenpalk",
    "Ma paberipuit","Ku paberipuit","Ks paberipuit","Hb paberipuit",
    "Küttepuit","Jäätmed"
]
wood_price_widgets = {name: bf(f"{name}:", 0) for name in price_names}

prices_grid = widgets.GridBox(
    children=list(wood_price_widgets.values()),
    layout=widgets.Layout(grid_template_columns='repeat(3, 350px)', grid_gap='6px')
)

upload_widget      = widgets.FileUpload(accept='.xlsx', multiple=False)
json_upload_widget = widgets.FileUpload(accept='.json', multiple=False)
calc_button        = widgets.Button(description="Arvuta")
output             = widgets.Output()


def _error(msg):
    display(widgets.HTML(f"<span style='color:#b00020;font-weight:600'>{msg}</span>"))

def _get_uploaded_content(upl):
    v = upl.value
    if not v:
        return None
    item = next(iter(v.values())) if isinstance(v, dict) else (v[0] if isinstance(v, (list, tuple)) else v)
    content = item.get('content') if isinstance(item, dict) else getattr(item, 'content', None)
    return content.tobytes() if isinstance(content, memoryview) else content

def _build_maht_hind_kokku_from_json(json_bytes):
    try:
        # Võta toorandmed
        if hasattr(json_bytes, "read"):
            raw = json_bytes.read()
        else:
            raw = json_bytes
        # memoryview -> bytes
        if isinstance(raw, memoryview):
            raw = raw.tobytes()
        # bytes / bytearray
        if isinstance(raw, (bytes, bytearray)):
            text = raw.decode("utf-8", errors="replace")
        # juba str
        elif isinstance(raw, str):
            text = raw
        else:
            # viimane katse
            text = bytes(raw).decode("utf-8", errors="replace")
        data = json.loads(text)
    except Exception as e:
        _error(f"JSON lugemise viga: {e}")
        return None
    stands = data.get('stands')
    if not stands:
        _error("JSON ei sisalda 'stands' massiivi.")
        return None

    rows = []
    for r in stands:
        area = r.get("total_area_ha", 0) or 0
        def rec(name_eng, h_key, d_key, v_key):
            h = r.get(h_key)
            d = r.get(d_key)
            v_ha = r.get(v_key)
            if v_ha is None:
                return
            rows.append({
                "Eraldise nr": r.get("stand_number"),
                "Puuliik": name_eng,
                "Kõrgus m": h,
                "Diameeter cm": d,
                "Pindala ha": area,
                "Tihedus m3/ha": v_ha,
                "Tagavara m3": area * v_ha
            })
        rec("Pine", "pine_bam_height_m", "pine_bam_dbh_cm", "pine_total_volume_m3_ha")
        rec("Spruce", "spruce_bam_height_m", "spruce_bam_dbh_cm", "spruce_total_volume_m3_ha")
        rec("Birch", "birch_bam_height_m", "birch_bam_dbh_cm", "birch_total_volume_m3_ha")
        rec("Other Deciduous", "other_deciduous_bam_height_m", "other_deciduous_bam_dbh_cm", "other_deciduous_total_volume_m3_ha")

    if not rows:
        _error("JSON-ist ei saadud ühtegi liikide rida.")
        return None

    df = pd.DataFrame(rows)

    # Liida nimetused (eeldab tree_name, relative_heights, log_volume_distribution on juba laaditud varasemates rakkudes)
    df = pd.merge(df, tree_name, left_on='Puuliik', right_on='Name_ENG', how='left')

    df['Suhteline tugikõrgus'] = np.nan
    for i, row in df.iterrows():
        name_ee = row['Name_EE']
        d = row['Diameeter cm']
        try:
            h24_val = relative_heights.loc[relative_heights['d'] == d, name_ee]
            if not h24_val.empty:
                df.at[i, 'Suhteline tugikõrgus'] = h24_val.values[0]
        except KeyError:
            pass

    def _calc_h24(r):
        if pd.notna(r['Kõrgus m']) and pd.notna(r['Suhteline tugikõrgus']) and r['Suhteline tugikõrgus'] != 0:
            val = int(np.ceil(r['Kõrgus m'] / r['Suhteline tugikõrgus']))
            return 16 if val < 16 else val
        return np.nan
    df['h24'] = df.apply(_calc_h24, axis=1).astype('Int64')

    def diameter_category(d):
        if pd.isna(d):
            return None
        if 5 <= d <= 52:
            return ((int(d) + 3) // 4) * 4
        if d > 52:
            return 52
        return None
    df['Diameetri klass'] = df['Diameeter cm'].apply(diameter_category).astype('Int64')

    df['Sortimendi jaotusklass'] = (
        df['Diameetri klass'].astype(str) + df['Name_EE'].astype(str) + df['h24'].astype(str)
    )

    merged = pd.merge(
        df,
        log_volume_distribution,
        left_on='Sortimendi jaotusklass',
        right_on='d klass+pl+h24 x m',
        how='inner'
    )

    for c in ['palk', 'peenp', 'paber', 'küte', 'jäätmed']:
        merged[c] = merged[c] * merged['Tagavara m3']

    merged = merged.drop(columns=['d klass+pl+h24 x m', 'kõrgus', 'kokku', 'Name_ENG'], errors='ignore')

    mappings = [
        ("Ma palk", ("MA","palk"), "Ma palk"),
        ("Ku palk", ("KU","palk"), "Ku palk"),
        ("Ks palk/pakk", ("KS","palk"), "Ks palk/pakk"),
        ("Teised liigid/Lv palk", ("LV","palk"), "Lv palk"),
        ("Ma peenpalk", ("MA","peenp"), "Ma peenpalk"),
        ("Ku peenpalk", ("KU","peenp"), "Ku peenpalk"),
        ("Ma paberipuit", ("MA","paber"), "Ma paberipuit"),
        ("Ku paberipuit", ("KU","paber"), "Ku paberipuit"),
        ("Ks paberipuit", ("KS","paber"), "Ks paberipuit"),
        ("Küttepuit", (None,"küte"), "Küttepuit"),
        ("Jäätmed", (None,"jäätmed"), "Jäätmed"),
    ]
    out_rows = []
    for sortiment, (name_ee, col), price_name in mappings:
        if name_ee:
            vol = merged.loc[merged["Name_EE"] == name_ee, col].sum()
        else:
            vol = merged[col].sum()
        out_rows.append({"Sortiment": sortiment, "Maht (tm)": vol, "Summa (€)": 0.0})
    mhk = pd.DataFrame(out_rows)
    mhk.loc[len(mhk)] = {
        "Sortiment": "Kokku",
        "Maht (tm)": mhk["Maht (tm)"].sum(),
        "Summa (€)": 0.0
    }
    return mhk

def arvuta(_):
    with output:
        clear_output()

        kulud = {
            "Kompleksteenus(€/tm)": komplekt_widget.value,
            "Transport (€/tm)": transport_widget.value,
            "Alghinna(%)": alghind_widget.value
        }
        if any(v < 0 for v in kulud.values()):
            _error("kulude väärtused ei tohi olla negatiivsed.")
            return

        # Hinnad
        xlsx_content = _get_uploaded_content(upload_widget)
        if xlsx_content is not None:
            wood_prices_in = pd.read_excel(io.BytesIO(xlsx_content))
            need_cols = {"Sortiment","Hind (€/tm)"}
            if not need_cols.issubset(wood_prices_in.columns):
                _error(f"Excel peab sisaldama veerge: {need_cols}")
                if show_details.value:
                    display(wood_prices_in.head())
                return
            wood_prices_in["Hind (€/tm)"] = pd.to_numeric(wood_prices_in["Hind (€/tm)"], errors="coerce")
            if wood_prices_in["Hind (€/tm)"].isna().any():
                _error("Veerus 'Hind (€/tm)' on mittearvulisi väärtusi.")
                if show_details.value:
                    display(wood_prices_in)
                return
            if (wood_prices_in["Hind (€/tm)"] < 0).any():
                _error("Hinnad ei tohi olla negatiivsed (Excel).")
                if show_details.value:
                    display(wood_prices_in[wood_prices_in["Hind (€/tm)"] < 0])
                return
            if show_details.value:
                display(wood_prices_in)
            prices = dict(wood_prices_in[["Sortiment","Hind (€/tm)"]].values)
        else:
            prices = {k: w.value for k, w in wood_price_widgets.items()}
            neg = {k:v for k,v in prices.items() if v < 0}
            if neg:
                _error("Hinnad ei tohi olla negatiivsed (manuaalne sisestus).")
                if show_details.value:
                    display(pd.DataFrame(list(neg.items()), columns=["Sortiment","Väärtus"]))
                return
            if show_details.value:
                display(pd.DataFrame(list(prices.items()), columns=["Sortiment","Hind (€/tm)"]))

        # JSON -> dünaamiline mahtude tabel või globaalne maht_hind_kokku
        json_content = _get_uploaded_content(json_upload_widget)
        if json_content is not None:
            mhk = _build_maht_hind_kokku_from_json(json_content)
            if mhk is None:
                return
        else:
            _error("Laadi üles katastri JSON, et arvutada maht ja tulud kokku.")
            return

        # Alias + normaliseerimine
        def _norm(s): return str(s).strip().lower()
        alias = {
            "teised liigid/lv palk": "lv palk"
        }
        prices_norm = {_norm(k): float(v) for k,v in prices.items()}

        mask = mhk["Sortiment"] != "Kokku"
        keys = mhk.loc[mask,"Sortiment"].map(_norm).map(lambda k: alias.get(k,k))
        price_series = keys.map(prices_norm).fillna(0.0)
        missing = sorted(set(keys[price_series.isna()].dropna()))
        if missing:
            _error("Puuduvad hinnad: " + ", ".join(missing) + " (kasutatakse 0).")
        price_series = price_series.fillna(0.0)
        mhk.loc[mask,"Summa (€)"] = mhk.loc[mask,"Maht (tm)"].values * price_series.values

        if "Kokku" in mhk["Sortiment"].values:
            idx = mhk.index[mhk["Sortiment"] == "Kokku"][0]
            mhk.at[idx, "Summa (€)"] = mhk.loc[mask, "Summa (€)"].sum()

        mhk["Maht (tm)"] = mhk["Maht (tm)"].round(1)
        mhk["Summa (€)"] = mhk["Summa (€)"].round(1)

        hind_kokku = float(mhk.loc[mhk["Sortiment"]=="Kokku","Summa (€)"].iloc[0])
        maht_kokku = float(mhk.loc[mhk["Sortiment"]=="Kokku","Maht (tm)"].iloc[0])
        maht_jaatmeteta = float(mhk.loc[~mhk["Sortiment"].isin(["Jäätmed","Kokku"]),"Maht (tm)"].sum())

        kulud_tm = kulud["Kompleksteenus(€/tm)"] + kulud["Transport (€/tm)"]
        kulud_jaatmeteta = maht_jaatmeteta * kulud_tm
        tulud_kulud_jaatmeteta = hind_kokku - kulud_jaatmeteta
        soovituslik_alghind = tulud_kulud_jaatmeteta * (1 - kulud["Alghinna(%)"]/100)

        if show_details.value:
            display(mhk[["Sortiment","Maht (tm)","Summa (€)"]])

        results = pd.DataFrame({
            "Väärtus":[maht_kokku, hind_kokku, kulud_jaatmeteta, tulud_kulud_jaatmeteta, soovituslik_alghind],
            "Ühik":["tm","€","€","€","€"]
        }, index=[
            "Maht kokku",
            "Tulud kokku",
            "Kulud (jäätmeteta)",
            "Tulud-Kulud (jäätmeteta)",
            "Soovituslik alghind"
        ])
        results["Väärtus"] = results["Väärtus"].round(1)
        display(results)

def _ensure_single_handler(btn, fn):
    for cb in list(getattr(btn._click_handlers, "callbacks", [])):
        btn.on_click(cb, remove=True)
    btn.on_click(fn)

_ensure_single_handler(calc_button, arvuta)

display(widgets.VBox([
    widgets.Label("Sisesta kulude väärtused:"),
    komplekt_widget, transport_widget, alghind_widget,
    widgets.Label("Sisesta puidu hinnad käsitsi või laadi üles Excel:"),
    prices_grid,
    widgets.Label("Või laadi üles hinnakiri (.xlsx) veergudega 'Sortiment' ja 'Hind (€/tm)':"),
    upload_widget,
    widgets.Label("Laadi üles katastri JSON (.json):"),
    json_upload_widget,
    show_details,
    calc_button,
    output
]))